In [4]:
# Create consolidated database with SQLite

import sqlite3

conn = sqlite3.connect('databases/graphs.db')
cursor = conn.cursor()

# Create graphs table
cursor.execute('''
CREATE TABLE IF NOT EXISTS graphs (
    id INTEGER PRIMARY KEY,
    graph6 TEXT NOT NULL,
    degree_seq TEXT NOT NULL,
    triangle_seq TEXT NOT NULL,
    num_nodes INTEGER NOT NULL,
    num_edges INTEGER NOT NULL,
    is_connected BOOLEAN,
    is_tree BOOLEAN,
    is_local BOOLEAN,
    is_global BOOLEAN,
    fer_group_encoded BLOB,
    UNIQUE(graph6))''')

# Index on degree_sequence and triangle_sequence for fast isomorphism checks.
# Two graphs have the same index iff networkx.fast_could_be_iso thinks they are the same.
cursor.execute('CREATE INDEX IF NOT EXISTS idx_fast_iso ON graphs(degree_sequence, triangle_sequence)')

conn.commit()
conn.close()

OperationalError: no such column: degree_sequence

In [3]:
# Add Marcos Laffitte's 1-10 amoeba graphs

# Header looks like (1, 0, 'conexa', 'ambas', '@')
import pickle
from database_handler import db_write, graph6_to_iso_invariants

with open("databases/MarcosLaffitte/Amoeba_Graphs_1_to_10_vertices_raw.pkl", "rb") as file:
  data = pickle.load(file)

conn = sqlite3.connect('databases/graphs.db')
cursor = conn.cursor()

for data_point in data:
  graph6  = data_point[4]
  num_nodes = data_point[0]
  num_edges = data_point[1]

  if data_point[2] == 'conexa':
    is_connected = True
  elif data_point[2] == 'disconexa':
    is_connected = False
  else:
    raise ValueError("is_connected:",data_point[2])

  if data_point[3] in ['local','ambas']:
    is_local = True
  else:
    is_local = False

  if data_point[3] in ['global','ambas']:
    is_global = True
  else:
    is_global = False

  is_tree = None

  degree_seq, triangle_seq = graph6_to_iso_invariants(graph6)

  db_write(conn, graph6, degree_seq, triangle_seq, num_nodes, num_edges,
           is_connected, is_tree, is_local, is_global,
           fer_group_encoded=None)

conn.close()

OperationalError: no such table: graphs

In [ ]:
# Add Marcos Laffitte's 11-22 amoeba trees

# Header looks like (1, 0, 'conexa', 'LA', '@')

conn = sqlite3.connect('databases/all_graphs.db')
cursor = conn.cursor()

with open("databases/MarcosLaffitte/Amoeba_Trees_1_to_22_vertices_raw.pkl", "rb") as file:
  data = pickle.load(file)

conn = sqlite3.connect('databases/graphs.db')
cursor = conn.cursor()

for data_point in data:
  # Filter small trees to avoid repetitions
  num_nodes = data_point[0]
  if num_nodes < 11:
    continue

  graph_g6  = data_point[4]
  num_edges = data_point[1]
  is_connected = True
  is_tree = True
  
  if data_point[3] == 'LA':
    is_local  = True
    is_global = True # Local trees are always global see Proposition 6 of seminal paper.
  elif data_point[3] == 'GA':
    is_global = True
    is_local  = False
  else:
    raise ValueError("is_amoeba:",data_point[3])

  degree_seq, triangle_seq = graph6_to_iso_invariants(graph6)

  db_write(conn, graph6, degree_seq, triangle_seq, num_nodes, num_edges,
           is_connected, is_tree, is_local, is_global,
           fer_group_encoded=None)

conn.close()

In [ ]:
# Add homeomorphically reduced 23-25 trees

